In [1]:
#based off crowsonkb's code

In [ ]:
!pip install accelerate peft

In [ ]:
!pip install bitsandbytes

In [1]:
import argparse
import json
from pathlib import Path
import sys

import accelerate
import peft
import torch
from torch import optim
from torch.utils import data
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import trange, tqdm

In [2]:
print = tqdm.external_write_mode()(print)


In [3]:
def batch_to_tensors(batch, device="cpu"):
    seq_len = max(len(x) for x in batch)
    input_ids = torch.zeros(len(batch), seq_len, dtype=torch.long, device=device)
    attention_mask = torch.zeros(len(batch), seq_len, dtype=torch.long, device=device)
    for i, x in enumerate(batch):
        input_ids[i, : len(x)] = torch.tensor(x, dtype=torch.long, device=device)
        attention_mask[i, : len(x)] = 1
    return input_ids, attention_mask

In [4]:
accelerator = accelerate.Accelerator(
        mixed_precision="bf16", gradient_accumulation_steps=1
    )

In [5]:
device = accelerator.device
is_main = accelerator.is_main_process
print0 = accelerator.on_main_process(print)

In [6]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit= True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

In [7]:
from huggingface_hub import login
login()

In [ ]:
pip install -U bitsandbytes accelerate

In [26]:
with accelerator.main_process_first():
    model_base = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-chat-hf",
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
accelerator.wait_for_everyone()

In [10]:
dropout = 0.0 #or 0.1

In [ ]:
# only relevant if you alr have model
# if args.start_from is not None:
#         print0(f"Loading adapter: {args.start_from}", file=sys.stderr)
#         with accelerator.main_process_first():
#             model = peft.PeftModel.from_pretrained(model_base, args.start_from, is_trainable=True)
#         if args.dropout is not None:
#             model.active_peft_config.lora_dropout = dropout

In [11]:
#otherwise, if starting from scratch
#intializing adapter
peft_config = peft.LoraConfig(
    peft.TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=8,
    lora_dropout=dropout,
)

In [12]:
#otherwise
model = peft.get_peft_model(model_base, peft_config)

In [13]:
accelerator.wait_for_everyone()


In [19]:
model.train()

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_proj): lora.Li

In [14]:
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [21]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


In [15]:
opt = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.99))

In [28]:
# import pandas
# dataset_train = pandas.read_csv('train_tokenized.csv')

In [31]:
# dataset_test = pandas.read_csv('test_tokenized.csv')

In [32]:
# dataset = dataset_train['tokenized_texts'].tolist() + dataset_test['tokenized_texts'].tolist()

In [16]:
import pickle

with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

In [ ]:
# dataset

In [32]:
dataloader = data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=batch_to_tensors,
    drop_last=True,
)

In [33]:
model, opt, dataloader = accelerator.prepare(model, opt, dataloader)

In [20]:
i = 0

In [29]:
from accelerate import Accelerator
import torch

accelerator = Accelerator()
device = accelerator.device
print(f"Using device: {device}")
print(f"Device count: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")


Using device: cuda
Device count: 4
Device 0: NVIDIA A100 80GB PCIe
Device 1: NVIDIA A100 80GB PCIe
Device 2: NVIDIA A100 80GB PCIe
Device 3: NVIDIA A100 80GB PCIe


In [ ]:
for epoch in trange(1):
    for input_ids, attention_mask in tqdm(dataloader):
        with accelerator.accumulate(model):
            outputs = model(
                input_ids[:, :-1],
                attention_mask=attention_mask[:, :-1],
                use_cache=False,
            )
            losses = torch.nn.functional.cross_entropy(
                outputs.logits.transpose(-1, -2),
                input_ids[:, 1:],
                reduction="none",
            )
            mask = attention_mask[:, :-1] * attention_mask[:, 1:]
            loss = torch.sum(losses * mask, dtype=torch.float32) / torch.sum(
                mask, dtype=torch.float32
            )

            accelerator.backward(loss)
            opt.step()
            opt.zero_grad()

            loss_global = accelerator.reduce(loss, "mean")
            print0(f"epoch: {epoch}, step: {i}, loss: {loss_global.item():g}")
            i += 1

  0%|          | 0/2754 [00:00<?, ?it/s]
                                        
  0%|          | 1/2754 [00:00<12:11,  3.76it/s]

epoch: 0, step: 205, loss: 3.66617



                                                
  0%|          | 2/2754 [00:00<12:57,  3.54it/s]

epoch: 0, step: 206, loss: 5.42273



                                                
  0%|          | 3/2754 [00:00<12:43,  3.60it/s]

...


                                                  
 30%|██▉       | 820/2754 [03:49<08:21,  3.86it/s]

epoch: 0, step: 1024, loss: 3.96136



                                                  
 30%|██▉       | 821/2754 [03:49<08:06,  3.97it/s]

epoch: 0, step: 1025, loss: 3.77831



                                                  
 30%|██▉       | 822/2754 [03:49<08:04,  3.99it/s]

In [39]:
def print_memory_usage():
    print(f"Allocated memory: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"Cached memory: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")

print_memory_usage()

Allocated memory: 73.42 GB
Cached memory: 76.90 GB


In [ ]:
accelerator.unwrap_model(model).save_pretrained("model", safe_serialization=True)